<a href="https://colab.research.google.com/github/SiPilip/pytorch_course/blob/main/04_pytorch_custom_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 04. PyTorch Custom Dataset

Kita sudah menggunakan beberapa dataset dengan pytorch before

Tetapi bagaiman jika kita ingin menggunakan data sendiri

Salah satu cara untuk melakukannya: Dataset kustom

## Domain Libraries

Bergantung pada apa yang kamu kerjakan, citra, teks, audio, kamu akan melihat tiap PyTorch domain libraries untuk data yang sudah ada berdasarkan fungsi dan kustom data loader


# 0. Import PyTorch dan setup agnostic device


In [1]:
import torch
from torch import nn

torch.__version__

'2.6.0+cu118'

In [2]:
# Setup perangkat agnostik
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
device

'cuda'

## 1. Ambil Data

Dataset kita adalah subset dari Food101 Dataset

Food101 dimulai dari 101 kelas berbeda dari makanan (750 Training, 250 Testing)

Dataset kita dimulai dari 3 kelas dan hanya 10% dari gambar (75 Training, 25 Testing)

Ketika mencoba pembelajaran mesil, ini adalah hal penting untuk melakukan dari kecil lalu meningkatkannya ketika dibutuhkan.

Intinya adalah seberapa cepat kita dapat bereksperiment


In [4]:
import requests
import zipfile
from pathlib import Path

# Buat jalur untuk folder data
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# Jika folder gambar tidak ada, download dan siapkan
if image_path.is_dir():
  print(f"{image_path} direktori sudah ada...")
else:
  print(f"{image_path} belum ada, membuat...")
  image_path.mkdir(parents=True, exist_ok=True)

data\pizza_steak_sushi direktori sudah ada...


In [ ]:
# Unduh pizza, steak, dan sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/data/pizza_steak_sushi.zip")
  print("Downloading pizza, steak sushi data")
  f.write(request.content)

# Unzip data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping data")
  zip_ref.extractall(image_path)

## 2. Menjadi satu dengan data (preparasi data, dan eksplorasi data)


In [8]:
import os
def walk_through_dir(dir_path):
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"Ada {len(dirnames)} direktori dan {len(filenames)} gambar didalam '{dirpath}'.")

In [ ]:
walk_through_dir(image_path)

In [ ]:
# Setup alur latih dan tes

train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

## 2.1 Visualisasikan Gambar

Yang akan kita buat

1. Ambil semua path gambar
2. Ambil gambar acak dengan random.choice()
3. Ambil kelas gamabr dengan `pathlib.Path.parent.stem`
4. Sejak kta bekerja dengan gambar, kita akan membuka gambar dengan PIL (pillow)
5. Kita akan menunjukkan gambar dan memberikan metadatanya


In [ ]:
import random

# Set seed
# random.seed(42)

# 1. Ambil jalur gambar
image_path_list = list(image_path.glob("*/*/*.jpg"))

# 2. Pilih jalur gambar secara acak
random_image_path = random.choice(image_path_list)
print(random_image_path)

# 3. Ambil kelas dari pathname (kelas gambar diambil dari direktori dimana gambarnya disimpan)
image_class = random_image_path.parent.stem
print(image_class)

In [ ]:
from PIL import Image

# 4. Buka gambar
img = Image.open(random_image_path)

# 5. Berikan metadata
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Ubah image ke array
img_as_array = np.asarray(img)

# Plot gambar
plt.figure(figsize=(10,7))
plt.imshow(img_as_array)
plt.title(f"Image class: {image_class} | Image shape: {img_as_array.shape}")
plt.axis(False)

## 3. Mengubah data

Sebelum kita dapat menggunakan gambar kita dengan PyTorch:

1. Ubah target data menjadi bentuk tensor (numerical representation dari gambar kita)
2. Ubah itu menjadi `torch.utils.data.Dataset` dan kemudian ubah menjadi `torch.utils.data.DataLoader` (untuk dibuatkan batch nya)


In [14]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 3.1 Transforming data dengan `torchvision.transforms`

Transform membantu gambar anda untuk bisa digunakan dengan model


In [15]:
# Buatkan transform untuk gambar
data_transform = transforms.Compose([
    # Resize gambar kita ke 64*64
    transforms.Resize(size=(64,64)),

    # Balikkan gambar secara acak dengan horizontal
    transforms.RandomHorizontalFlip(p=0.5),

    # Ubah gambar menjadi torch.Tensor
    transforms.ToTensor()
])

In [ ]:
data_transform(img)[:2]

In [ ]:
def plot_transformed_images(image_paths, transform, n=3, seed=42):
  if seed:
    random.seed(seed)

  random_image_paths = random.sample(image_paths, k=n)
  for image_path in random_image_paths:
    with Image.open(image_path) as f:
      fig, ax = plt.subplots(nrows=1, ncols=2)
      ax[0].imshow(f)
      ax[0].set_title(f"Original\nSize: {f.size}")
      ax[0].axis(False)

      # Transform and plot target gambar
      transformed_image = transform(f).permute(1, 2, 0) # Kita akan ubah bentuk untuk matplotlib
      ax[1].imshow(transformed_image)
      ax[1].set_title(f"Transformed\nShape: {transformed_image.shape}")
      ax[1].axis("off")

      fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)

plot_transformed_images(image_paths=image_path_list,
                        transform=data_transform,
                        n=3,
                        seed=None)


## 4. Opsi 1: Muat data gambar dengan `ImageFolder`

Kita bisa muat klasifikasi gambar dengan menggunakan `torchvision.datasets.ImageFolder`


In [ ]:
# Gunakan ImageFolder untuk membuat dataset
from torchvision import datasets
train_data = datasets.ImageFolder(root=train_dir,
                                  transform=data_transform,
                                  target_transform=None)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)

train_data, test_data

In [ ]:
# Ambil nama kelas dengan list
class_names = train_data.classes
class_names

In [ ]:
# Ambil nama kelas sebagai dict
class_dict = train_data.class_to_idx
class_dict

In [ ]:
# Cek panjang dari dataset
len(train_data), len(test_data)

In [ ]:
train_data.samples[:5], train_data.targets[:5]

In [ ]:
# Index didalam train_data dataset untuk mendapatkan satu gambar
img, label = train_data[0][0], train_data[0][1]
img, class_names[label]
print(f"Gambar tenspr:\n {img[:2]}")
print(f"Bentuk gambar:\n {img.shape}")
print(f"Gambar datatype: \n {img.dtype}")
print(f"Label gambar: \n {label}")
print(f"Label gambar: \n {type(label)}")

In [ ]:
# Mengatur ulang dimensi -> [H, W, C]
img_permute = img.permute(1,2,0)

# Lihatkan shape berbeda
print(f"Bentuk asli: {img.shape} -> [C, H, W]")
print(f"Permutasi Gambar: {img_permute.shape} -> [H, W, C]")

# Plot gambar
plt.figure(figsize=(5,5))
plt.imshow(img_permute)
plt.axis("off")
plt.title(class_names[label], fontsize=20)

## 4.1 Ubah gambar termuat menjadi `DataLoader`

Sebuah `DataLoader` akan membantu kita mengubah dataset menjadi iterable dan kita dapat mengkostumisasi `batch_size` jadi model kita dapat melihat `batch_size` gambar dalam satu waktu


In [ ]:
import os
os.cpu_count()

In [ ]:
# Ubah data latih dan data test ke DataLoader
from torch.utils.data import DataLoader

BATCH_SIZE = 1

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=1,
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             num_workers=1,
                             shuffle=False)

train_dataloader, test_dataloader

In [ ]:
img, label = next(iter(train_dataloader))

print(f"Bentuk gambar: {img.shape} -> [Batch size, C, H, W]")
print(f"Label shape {label.shape}")

In [ ]:
!python --version

## 5. OPsi 2: Muat Gambar dengan Kustom `Datase`

1. Bisa mengambil gambar dari file
2. Bisa mengambil kelas dari Dataset
3. Bisa mengambil kelas sebagai `dictionary` dari dataset

Pros:

- Bisa membuat dataset hampir dari papaun
- Tidak dibatasi oleh PyTorch

Cons:

- Walaupun kita dapat membuat Dataset dari apapun, itu bukan berarti akan dapat digunakan
- Mengguanakan Dataset kustom seringkali dapat memberikan masalah performa


In [29]:
import os
import pathlib
import torch

from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from typing import Tuple, Dict, List

In [ ]:
# Instans dari torchvision.dataseats.ImageFolder()
train_data.classes, train_data.class_to_idx

### 5.1 Buat helper function untuk mendapatkan nama kelas

Kita akan mendapatkan fungsi untuk:

1. Ambil nama kelas menggunakan `os.scandir()` untuk melintasi target direktori
2. Mengeluarkan output kesalahan jika nama kelas tidak ditemukan
3. Ubah nama kelas menjadi dict dan list lalu return mereka


In [ ]:
# Setup path untuk target directory
target_directory = train_dir
print(f"Target directory: {target_directory}")

# Ambil nama kelas dari target directory
class_name_found = sorted([entry.name for entry in list(os.scandir(target_directory))])
class_name_found

In [32]:
def find_classes(directory: str) -> Tuple[List[str], Dict[str, int]]:
  """Cari folder kelas dalam target direktori."""

  # 1. Ambil nama kelas dari scanning target direktori
  classes = sorted([entry.name for entry in os.scandir(directory) if entry.is_dir()])

  # 2. Berikan error jika nama kelas tidak ditemukan
  if not classes:
    raise FileNotFoundError(f"Tidak dapat menemukan kelas di {directory}")

  # 3. Buat dictoanary dari index label
  class_to_idx = {class_name: i for i, class_name in enumerate(classes)}

  return classes, class_to_idx

In [ ]:
find_classes(target_directory)

### 5.2 Buat `Dataset` custom untuk replikasi `ImageFolder`

Untuk membuat custom dataset kita sendiri, kita harus:

1. Membuat subclass dari `torch.utils.data.Dataset`
2. Inisiasi subclass kita dengan target direktori, lalu kita akan mentransform datanya
3. Buat atribut terpisah

- paths - path dari gambar kita
- transform - transform yang akan kita gunakan
- classes - list dari target kelas kita
- class_to_idx - dict dari target kelas yang di map ke label integer

4. Buat function untuk `load.images()`
5. Menulis ulang `__len__()` untuk mereturn panjang dari dataset kita
6. Menulis ulang `__getitem()__` untuk mereturn sample


In [34]:
# Buat kelas custom dataset
from torch.utils.data import Dataset

# 1. Subclass torch.tuils.data.Dataset
class ImageFolderCustom(Dataset):
  # 2. Inisialisasi dataset kustom
  def __init__(self,
               targ_dir: str,
               transform=None):
    # Buat atribut kelas
    self.paths = list(pathlib.Path(targ_dir).glob("*/*.jpg"))

    # Setup transforms
    self.transform = transform

    # Buat classes dan class_to_idx
    self.classes, self.class_to_idx = find_classes(targ_dir)

  # 4. Buat function untuk memuat gambar
  def load_image(self, index: int) -> Image.Image:
    image_path = self.paths[index]
    return Image.open(image_path)

  # 5. Overwrite __len__()
  def __len__(self) -> int:
    return len(self.paths)

  # 6. Overwrite __getitem__() untuk mereturn sample
  def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
    img = self.load_image(index)
    class_name = self.paths[index].parent.name # Mengharapkan path dalam format: /data_folder/class_name/name.jpg
    class_idx = self.class_to_idx[class_name]

    # Transforms jika diperlukan
    if self.transform:
      return self.transform(img), class_idx
    else:
      return img, class_idx

In [35]:
# Buat transform
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

In [36]:
# Test out ImageFolderCustom
train_data_custom = ImageFolderCustom(targ_dir=train_dir,
                                      transform=train_transforms)
test_data_custom = ImageFolderCustom(targ_dir=test_dir,
                                     transform=test_transforms)

In [ ]:
train_data_custom, test_data_custom

In [ ]:
len(train_data), len(train_data_custom)

In [ ]:
train_data_custom.classes

In [ ]:
train_data_custom.class_to_idx

In [ ]:
# Periksa kesamaan antara ImageFolder Dataset dan ImageFolderCustomDataset
print(train_data_custom.classes == train_data.classes)
print(test_data_custom.classes == test_data.classes)

# 5.3 Buat fungsi untuk menampilkan gambar acak

1. Ambil `Dataset` dan beberapa parameter lain, seperti nama kelas, dan berapa banyak gambar yang akan ditampilkan
2. Untuk mencegah gambar ditampilkan banyak, kita batasi 10 gambar
3. Buat random seed untuk reproducibility
4. Ambil list dari sample acak untuk target dataset
5. Setup `matplotlib.plot`
6. Buat perulangan dari gambar acak lalu tampilkan dengan matplotlib
7. Pastikan dimensi nya [H, W, C]


In [42]:
# 1. Buat fungsi untuk mengambil dataset
def display_random_images(dataset: torch.utils.data.Dataset,
                          classes: List[str] = None,
                          n: int = 10,
                          display_shape: bool = True,
                          seed: int = None):
  # 2. Sesuaikan tampilan jika n nya terlalu ringgi
  if n > 10:
    n = 10
    display_shape = False
    print(f" Untuk kepentingan gambar, n tidak boleh lebih besar dari 10")

  # 3. Atur seed
  if seed:
    random.seed(seed)

  # 4. Ambil index sample random
  random_samples_idx = random.sample(range(len(dataset)), k=n)

  # 5. Buat plot
  plt.figure(figsize=(16,8))

  # 6. Lakukan perulangan index acak dan plot dengan matplotlib
  for i, targ_sample in enumerate(random_samples_idx):
    targ_image, targ_label = dataset[targ_sample][0], dataset[targ_sample][1]

    # 7. Sesuaikan dimensi tensor untuk plotting
    targ_image_adjust = targ_image.permute(1,2,0) # [C, H, W] -> [H, W, C]

    # Plot smaple yang sudah di atur dimensinya
    plt.subplot(1, n, i+1)
    plt.imshow(targ_image_adjust)
    plt.axis("off")

    # Tambahkan label jika diperlukan
    if classes:
      title = f"Class: {classes[targ_label]}"
      if display_shape:
        title = title + f"\nShape: {targ_image_adjust.shape}"
      plt.title(title)


In [ ]:
# Tampilkan gambar acak dari ImageFolder
display_random_images(train_data,
                      n=5,
                      classes=class_names,
                      seed=None)

In [ ]:
# Tampilkan gambar acak dari ImageFoldeCustom dataset
display_random_images(train_data_custom,
                      n=20,
                      classes=class_names,
                      seed=42)


### 5.4 Ubah custom muat gambar menjadi DataLoader's


In [45]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
NUM_WORKERS = 0

train_dataloader_custom = DataLoader(dataset=train_data_custom,
                                     batch_size=BATCH_SIZE,
                                     num_workers = NUM_WORKERS,
                                     shuffle=True)
test_dataloader_custom = DataLoader(dataset=test_data_custom,
                                    batch_size=BATCH_SIZE,
                                    num_workers=NUM_WORKERS,
                                    shuffle=False)

In [ ]:
train_dataloader_custom, test_dataloader_custom

In [ ]:
img_custom, label_custom = next(iter(train_dataloader_custom))

img_custom.shape, label_custom.shape

## 6. Bentuk lain dari transforms (Augmentasi Data)


Data augmentasi adalah proses artifisial untuk menamahkan keunikan dari data latih

Untuk kasus ini. berarti memberikan ragam bentuk lain dari gambar kita

Mari melihat, satu jenis augmentasi data yang digunakan untuk melatih PyTorch vision model


In [48]:
# Mari lihat tirvialaugment
from torchvision import transforms
train_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31), # -> 0-31 seberapa intens ini dilakukan
    # transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
])

In [49]:
# Ambil semua image paths
image_path_list = list(image_path.glob("*/*/*.jpg"))

In [ ]:
plot_transformed_images(
    image_paths=image_path_list,
    transform=train_transform,
    n=3,
    seed=None
)

## 7. Model 0: TinyVGG tanpa data augmentasi

Mari mereplikasi TinyVGG arsitektur dari CNN Explainer


### 7.1 Buat transforms dan muat data untuk Model 0


In [51]:
# Buat simple transform
simple_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
    ])

In [52]:
# 1. Load dan transformasi data
from torchvision import datasets
train_data_simple = datasets.ImageFolder(root=train_dir,
                                         transform=simple_transform)
test_data_simple = datasets.ImageFolder(root=test_dir,
                                         transform=simple_transform)

# 2. Ubah dataset menjadi DataLoaders
import os
from torch.utils.data import DataLoader

# Setup batch
BATCH_SIZE = 32
NUM_WORKERS = 0

# Create DataLoader's
train_dataloader_simple = DataLoader(dataset=train_data_simple,
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     num_workers=NUM_WORKERS,
                                     )
test_dataloader_simple = DataLoader(dataset=test_data_simple,
                                     batch_size=BATCH_SIZE,
                                     shuffle=False,
                                     num_workers=NUM_WORKERS,
                                     )

### 7.2 Buat TinyVGG Class


In [53]:
class TinyVGG(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_units: int,
               output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                      stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                      stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )

  def forward(self, x):
    x = self.conv_block_1(x)
    # print(x.shape)
    x = self.conv_block_2(x)
    # print(x.shape)
    x = self.classifier(x)
    return x

In [ ]:
torch.manual_seed(42)
model_0 = TinyVGG(input_shape=3,
                  hidden_units=10,
                  output_shape=len(class_names)).to(device)
model_0

## 7.3 Coba forward pass pada satu gambar


In [55]:
# Ambil satu batch data
image_batch, label_batch = next(iter(train_dataloader_simple))

In [ ]:
# Forward pass
model_0(image_batch.to(device))

### 7.4 Gunakan `torchinfo` untuk mendapatkan bentuk dari model


In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model_0, input_size=[1,3,64,64])

## 7.5 Buat fungsi proses latih dan tes

- `train_step()` - mengambil model dan dataloader untuk melatihan model di dataloader
- `test_step()` - mengambil model dan dataloader untuk mengevaluasi dan menguji model


In [59]:
# Create train_step()
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device=device):
  # Letakkan model dalam mode latih
  model.train()

  # Setup train loss dan akurasi
  train_loss, train_acc = 0,0

  # Loop
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. Kalkulasi loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Kalkulasi metrik akurasi
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class==y).sum().item() / len(y_pred)

  # Sesuaikan metrik utnuk mendapatkan rearata loss dan akurasi tiap batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [60]:
# Buat proses test
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device=device):
  # Letakkan model pada mode valuasi
  model.eval()

  # Buat test loss dan test akurasi
  test_loss, test_acc = 0, 0

  # Nyalakan inference mode
  with torch.inference_mode():
    for batch, (X,y) in enumerate(dataloader):
      # Kirim data ke target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)

      # 2. Kalkulasi loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      # Kalkulasi akurasio
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))

    # Buat matrik untuk rerata loss dan akurasi tiap batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

## 7.6 Buat fungsi `train()` dan kombinasikan dengan `train_step()` dan `test_step()`


In [ ]:
from tqdm.auto import tqdm
 # 1. Buat fungsi train yang akan mengambil banyak macam dari parameter model + optimizer + data loader
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          device=device):

  # 2. Buat dictionary dengan results kosong
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}

  # 3. Lakukan perulangan dalam fungsi training dan testing sesuai jumlah epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    # 4. Visualisasi
    print(f"Epoch: {epoch} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

    # 5. Update diktionari hasil
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # 6. Return the fillder results
  return results

## 7.7 Latih dan evaluasi model 0


In [62]:
# Buat random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Atur jumlah epochs
NUM_EPOCHS = 5

# Buat ulang instansi dari TinyVGG
model_0 = TinyVGG(input_shape=3,
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)

# Setup loss function dan optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(),
                             lr=0.001)

In [ ]:
!pip install timeit

In [ ]:
from timeit import default_timer as timer
start_time = timer()

# Latih model_0
model_0_results = train(model=model_0,
                        train_dataloader=train_dataloader_simple,
                        test_dataloader=test_dataloader_simple,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)

# Akhiri timer dan print berapa waktu yang dibutuhkan
end_time = timer()
print(f"Total training time:  {end_time-start_time:.3f} seconds")



### 7.8 Plot kurva loss pada model_0 kita

Sebuah kurva loss dapat mentracking progress model kita dari tiap waktu


In [ ]:
# Ambil results key dari model_0
model_0_results.keys()

In [66]:
def plot_loss_curves(results: Dict[str, List[float]]):
  # Ambil niai loss dari results
  loss = results["train_loss"]
  test_loss = results["test_loss"]

  # Ambil nilai akurasi dari results
  accuracy = results["train_acc"]
  test_accuracy = results["test_acc"]

  # Tampilkan keadaan epoch
  epochs = range(len(results["train_loss"]))

  # Buat plot
  plt.figure(figsize=(15,7))

  # Plot loss nya
  plt.subplot(1,2,1)
  plt.plot(epochs, loss, label="train_loss")
  plt.plot(epochs, test_loss, label="test_loss")
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.legend()

  # Plot akurasi
  plt.subplot(1,2,2)
  plt.plot(epochs, accuracy, label="train_accuracy")
  plt.plot(epochs, test_accuracy, label="test_accuracy")
  plt.title("Accuracy")
  plt.title("Epochs")
  plt.legend()


In [ ]:
plot_loss_curves(model_0_results)

## 8. Bagaimana seharusnya kurva loss kita dibentuk?

Kurva loss adalah salah satu cara keren untuk menuntaskan permasalahan dalam model

- Underfitting - ketika model kita lossnyya masih terlalu tinggi, masish bisa menjadi lebih kecil
- Overfitting - ketika loss train memiliki range jauh dengan loss test
- Mantap - ketika range loss train dan test berdekatan


# Untuk improve overfitting

1. Ambil lebih banak data
2. Gunakan data augmentasi - agar data train lebih susah untuk mempelajari gambar
3. Gunakan dataset yang lebih baik
4. Gunakan transfer learning
5. Simpelkan model kita, terkadang model yang kompleks belum berarti bisa menjadi efektif
6. Gunakan learning rate decay - memperkecil learning rate dari waktu ke waktu


## 9. Model 1: TinyVGG dengan Augmentasi Data

Mari mencoba model lain, kali ini menggunakan model yang sama tetapi dengan menggunakan augmentasi data


### 9.1 Membuat transform dengan augmentasi data


In [68]:
# Membuat transform data latih dengan TrivialAugment
from torchvision import transforms
train_transform_trivial = transforms.Compose([
  transforms.Resize(size=(64,64)),
  transforms.TrivialAugmentWide(num_magnitude_bins=31),
  transforms.ToTensor()
])
test_transform_simple = transforms.Compose([
  transforms.Resize(size=(64,64)),
  transforms.ToTensor()
])

### 9.2 Membuat data train dan test `DataLoader` serta `Dataset` Data Augmentasi


In [69]:
# Ubah folder gamabr menjadi Datasets
from torchvision import datasets
train_data_augmented = datasets.ImageFolder(root=train_dir,
                                            transform=train_transform_trivial)
test_data_augmented = datasets.ImageFolder(root=test_dir,
                                           transform=test_transform_simple)

In [86]:
# Mengubah datasets menjadi dataloaders
import os
from torch.utils.data import DataLoader

BATCH_SIZE = 32
NUM_WORKERS = 0

torch.manual_seed(42)
train_dataloader_augmented = DataLoader(dataset=train_data_augmented,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                        num_workers=NUM_WORKERS)

test_dataloader_simple = DataLoader(dataset=test_data_simple,
                                    batch_size=BATCH_SIZE,
                                    shuffle=True,
                                    num_workers=NUM_WORKERS)

### 9.3 Rancang dan latih model 1

Kali ini kita akan membuat arsitektur model kecuali kali ini kita menggunakan data augmentasi dari data latih


In [ ]:
# Buat model_1 dan kirim ke target device
torch.manual_seed(42)
model_1 = TinyVGG(input_shape=3,
                  hidden_units=10,
                  output_shape=len(train_data_augmented.classes)).to(device)
model_1

Keren! Sekarang kita punya model dan dataloaders, kita buat loss function dan optimizer dan memanggil `train()`


In [ ]:
# Atur random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Atur jumlah epochs
NUM_EPOCHS = 5

# Buat loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model_1.parameters(),
                             lr=0.001)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model 1
model_1_results = train(model=model_1,
                        train_dataloader=train_dataloader_custom,
                        test_dataloader=test_dataloader_simple,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS,
                        device=device)

# Akhiri timer
end_time = timer()
print(f"Total waktu yang dibutuhkan untuk melatih model_1: {end_time-start_time:.3f} seconds")

In [ ]:
model_1_results

### 9.4 Plot kurva loss dari model 1

Kurva loss membantu kita mengevaluasi performa model dari waktu ke waktu


In [ ]:
plot_loss_curves(model_1_results)

## 10. Bandingkan hasil model

Setelah mengevaluasi hasil eksperimen model, sangat penting untuk membandingkan model yang sudah kita buat.

Ada beberapa cara untuk melakukannya

1. Hard coding (yang akan kita lakukan)
2. PyTorch + Tensorboard
3. Weights & Biases
4. MLFlow


In [ ]:
# Lihat jika paket sudah terinstall didalam google collab
try:
  import pandas as pd
except:
  %pip install pandas

In [ ]:
import pandas as pd
model_0_df = pd.DataFrame(model_0_results)
model_1_df = pd.DataFrame(model_1_results)

model_0_df, model_1_df